# Finance Data OS — Week 3 Notebook

Pulled 5+ years of real history (**AAPL, MSFT, NVDA, TSLA**) with `yfinance` → **Validation** → **Mart** → **Power BI**.

**Run order:** Imports → Step 1 → Step 2 → Step 3 → Step 4 → Step 5 (optional charts)  
> Uses `yfinance` with `auto_adjust=True` so **close is already adjusted** (no `Adj_Close`).


## Setup — Imports & Paths


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import yfinance as yf

# --- User settings ---
TICKERS = ["AAPL", "MSFT", "NVDA", "TSLA"]
DATE_START = "2019-01-01"
DATE_END   = "2025-12-31"

# Lake & mart locations
LAKE_ROOT = Path("lake/ohlcv")
MART_PATH = Path("lake/feature_mart.parquet")

LAKE_ROOT.mkdir(parents=True, exist_ok=True)
MART_PATH.parent.mkdir(parents=True, exist_ok=True)

print("LAKE_ROOT:", LAKE_ROOT.resolve())
print("MART_PATH:", MART_PATH.resolve())

REQUIRED = ["date","ticker","open","high","low","close","volume"]


## Helpers — normalize & types


In [ ]:
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Lowercase, flatten any MultiIndex, and map common aliases."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ["/".join([str(x) for x in c if str(x) != "None"]) for c in df.columns]
    df.columns = [str(c).strip().lower().replace(" ", "_") for c in df.columns]
    # alias
    alias = {"symbol": "ticker", "adj_close": "close"}
    df = df.rename(columns=alias)
    return df

def coerce_types(df: pd.DataFrame) -> pd.DataFrame:
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"], errors="coerce")
    for c in ["open","high","low","close","volume"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


## Step 1 — Pull & Save OHLCV (adjusted)


In [ ]:
def pull_ohlcv_adjusted(ticker: str, start: str, end: str) -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    df = df.reset_index().rename(columns={
        "Date":"date","Open":"open","High":"high","Low":"low","Close":"close","Volume":"volume"
    })
    df = normalize_columns(df)
    df["ticker"] = ticker
    df = coerce_types(df)
    df = df[REQUIRED]
    return df

def write_partitioned(df: pd.DataFrame, ticker: str) -> None:
    for y in sorted(df["date"].dt.year.unique()):
        out_dir = LAKE_ROOT / f"ticker={ticker}" / f"year={y}"
        out_dir.mkdir(parents=True, exist_ok=True)
        (df[df["date"].dt.year == y]
            .to_parquet(out_dir / f"{ticker}_{y}.parquet", index=False))
    print(f"✔ lake write complete for {ticker}")

for t in TICKERS:
    print(f"↻ pulling {t}")
    _df = pull_ohlcv_adjusted(t, DATE_START, DATE_END)
    write_partitioned(_df, t)

print("✅ Step 1 complete")


## Step 2 — Repair Legacy Files (safe to re-run)


In [ ]:
files = list(LAKE_ROOT.glob("ticker=*/year=*/*.parquet"))
print("files to inspect:", len(files))

for p in files:
    df = pd.read_parquet(p)
    df = coerce_types(normalize_columns(df))
    part_ticker = p.parent.parent.name.split("=")[1]
    part_year   = int(p.parent.name.split("=")[1])

    if "ticker" not in df.columns:
        df["ticker"] = part_ticker
    df["ticker"] = df["ticker"].fillna(part_ticker)

    if all(c in df.columns for c in REQUIRED):
        df = df[REQUIRED]

    df = df[(df["ticker"] == part_ticker) & (df["date"].dt.year == part_year)]
    df.to_parquet(p, index=False)

print("✅ repair pass complete (if needed)")


## Step 3 — Validate Lake


In [ ]:
issues = []
files = list(LAKE_ROOT.glob("ticker=*/year=*/*.parquet"))
if not files:
    raise AssertionError("No parquet files found under lake/ohlcv.")

for p in files:
    part_ticker = p.parent.parent.name.split("=")[1]
    part_year   = int(p.parent.name.split("=")[1])

    df = coerce_types(normalize_columns(pd.read_parquet(p)))

    miss = [c for c in REQUIRED if c not in df.columns]
    if miss:
        issues.append((str(p), f"missing cols: {miss}"))
        continue

    nulls = df[REQUIRED].isna().sum()
    if (nulls > 0).any():
        issues.append((str(p), f"nulls: {nulls[nulls>0].to_dict()}"))

    for c in ["open","high","low","close","volume"]:
        if not pd.api.types.is_numeric_dtype(df[c]):
            issues.append((str(p), f"non-numeric {c}: {df[c].dtype}"))

    if df.duplicated(subset=["ticker","date"]).any():
        issues.append((str(p), "duplicate (ticker,date) rows"))

    if (df["ticker"] != part_ticker).any():
        issues.append((str(p), f"wrong ticker values (expected {part_ticker})"))
    if (df["date"].dt.year != part_year).any():
        issues.append((str(p), f"wrong year values (expected {part_year})"))

if issues:
    print("❌ Validation issues:")
    for where, msg in issues:
        print(f" - [{where}] {msg}")
    raise AssertionError(f"Validation failed: {len(issues)} issue(s).")
else:
    print(f"✅ validation passed ({len(files)} files)")


## Step 4 — Build Feature Mart


In [ ]:
def build_feature_mart() -> pd.DataFrame:
    dfs = []
    for fp in LAKE_ROOT.glob("ticker=*/year=*/*.parquet"):
        part_ticker = fp.parent.parent.name.split("=")[1]
        part_year   = int(fp.parent.name.split("=")[1])

        df = coerce_types(normalize_columns(pd.read_parquet(fp)))
        df = df.sort_values(["ticker","date"]).reset_index(drop=True)

        r1    = df["close"].pct_change().reset_index(drop=True)
        sma10 = df["close"].rolling(10, min_periods=10).mean().reset_index(drop=True)
        vol20 = df["close"].rolling(20, min_periods=20).std(ddof=0).reset_index(drop=True)

        if len(r1) != len(df) or len(sma10) != len(df) or len(vol20) != len(df):
            raise AssertionError(
                f"Index mismatch in partition {part_ticker}-{part_year}: "
                f"len(df)={len(df)} r1={len(r1)} sma10={len(sma10)} vol20={len(vol20)}"
            )

        df["return1"] = r1
        df["sma10"]   = sma10
        df["vol20"]   = vol20
        df = df.dropna(subset=["return1","sma10","vol20"])

        dfs.append(df[["date","ticker","close","return1","sma10","vol20"]])

    mart = pd.concat(dfs, ignore_index=True).sort_values(["ticker","date"]).reset_index(drop=True)

    if mart.isna().any().any():
        raise AssertionError("Feature mart still contains nulls.")
    if mart.duplicated(subset=["ticker","date"]).any():
        raise AssertionError("Duplicate (ticker,date) rows detected in mart.")

    return mart

mart_df = build_feature_mart()
mart_df.to_parquet(MART_PATH, index=False)
print("✅ Feature mart saved:", MART_PATH, "rows:", len(mart_df))
mart_df.head()


## Step 5 — Sanity Check & (optional) quick charts


In [ ]:
for t in TICKERS:
    d = mart_df[mart_df["ticker"]==t]
    print(f"\n{t}: {len(d)} rows {d['date'].min().date()} → {d['date'].max().date()}")
    print(d.head(2)[["date","close","return1","sma10","vol20"]])
    print(d.tail(2)[["date","close","return1","sma10","vol20"]])

# Optional inline chart (uncomment to show)
# import matplotlib.pyplot as plt
# base = mart_df.groupby("ticker")["close"].transform("first")
# mart_df.assign(norm_close=mart_df["close"]/base).pivot(index="date", columns="ticker", values="norm_close").plot(figsize=(10,4))
# plt.title("Normalized Close (start = 1.0)")
# plt.ylabel("Normalized")
# plt.tight_layout()
# plt.show()

print("\n🎉 All steps complete.")
